In [1]:
# python required >= 3.5 version
import sys
assert sys.version_info >=(3,5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

import numpy as np
import os
import pandas as pd
# make the output satable accros run
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

ROOT = "."
chapter_id = "DNN"
image_path = os.path.join(ROOT,"images",chapter_id)
os.makedirs(image_path,exist_ok=True)

def save_fig(fig_id,tigh_layout=True,fig_extension="png",resolution=300):
    path = os.path.join(image_path,fig_id + "." + fig_extension)
    print("save figure ",fig_id)
    if tigh_layout:
        plt.tight_layout()
    plt.savefig(path,format=fig_extension,dpi=resolution)


In [2]:
# get dataset fashion_mnist dataset
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()
# standarisasi train data
X_train_full = X_train_full/255.0

# split train data to train and validasi data
X_valid,X_train = X_train_full[:5000], X_train_full[5000:]
y_valid,y_train = y_train_full[:5000], y_train_full[5000:]

# l1 and l2 regulazation

In [3]:
layer = keras.layers.Dense(100,activation="elu",
                           kernel_initializer = "he_normal",
                           kernel_regularizer=keras.regularizers.l2(1e-2))
# or l1(0.1) for ℓ1 regularization with a factor of 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

In [4]:
model_1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300,activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(1e-2)),
    
    keras.layers.Dense(100,activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(1e-2)),

    keras.layers.Dense(10,activation="softmax",
                       kernel_initializer="glorot_uniform",
                       kernel_regularizer=keras.regularizers.l2(1e-2))
    
])

In [5]:
model_1.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model_1.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 6s 3ms/step - loss: 1.5822 - accuracy: 0.7771 - val_loss: 0.9151 - val_accuracy: 0.7924
Epoch 2/2
1719/1719 [==============================] - 5s 3ms/step - loss: 0.9106 - accuracy: 0.7930 - val_loss: 0.8682 - val_accuracy: 0.8088


pada saat program diatas terjadi boiler code pengulangan berapa layer yang sama dapat di hilangkan dengan simple menggunakan from functools import partial

In [6]:
from functools import partial
RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(1e-2))
    

In [7]:
model_2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    # RegularizedDense(10, activation="softmax",kernel_initializer="glorot_uniform")
    keras.layers.Dense(10, activation="softmax",kernel_initializer="glorot_uniform")
])
model_2.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model_2.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 5s 3ms/step - loss: 1.3988 - accuracy: 0.7805 - val_loss: 0.8320 - val_accuracy: 0.7812
Epoch 2/2
1719/1719 [==============================] - 4s 3ms/step - loss: 0.6907 - accuracy: 0.8095 - val_loss: 0.6699 - val_accuracy: 0.8054


In [8]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 300)               235500    
                                                                 
 dense_5 (Dense)             (None, 100)               30100     
                                                                 
 dense_6 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


# Regular Dropout

In [23]:
y_train.max()

9

In [10]:
tf.random.set_seed(42)
np.random.seed(42)
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

In [26]:
model_3 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dropout(rate=2e-1),
    keras.layers.Dense(300,activation="elu",kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=2e-1),
    keras.layers.Dense(100,activation="elu",kernel_initializer = "he_normal"),
    keras.layers.Dropout(rate=2e-1),
    keras.layers.Dense((y_train.max()+1),activation="softmax",kernel_initializer="glorot_normal")
    
])

model_3.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.Nadam(learning_rate=1e-3,beta_1=0.9,beta_2=0.999),
                metrics=["accuracy"])
n_epochs=5
per_lr_sc = keras.callbacks.ReduceLROnPlateau(factor=5e-1, patience=5)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
check_point = keras.callbacks.ModelCheckpoint("droput_reguler.h5",save_best_only=True)
history_3 = model_3.fit(X_train,y_train,epochs=n_epochs,
                        validation_data = [X_valid,y_valid],
                        callbacks=[per_lr_sc,tensorboard_cb,check_point]) 

Epoch 1/5
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5819 - accuracy: 0.7886 - val_loss: 0.4439 - val_accuracy: 0.8346 - lr: 0.0010
Epoch 2/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4570 - accuracy: 0.8311 - val_loss: 0.3504 - val_accuracy: 0.8688 - lr: 0.0010
Epoch 3/5
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4221 - accuracy: 0.8419 - val_loss: 0.3573 - val_accuracy: 0.8708 - lr: 0.0010
Epoch 4/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4002 - accuracy: 0.8516 - val_loss: 0.3257 - val_accuracy: 0.8778 - lr: 0.0010
Epoch 5/5
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3844 - accuracy: 0.8571 - val_loss: 0.3219 - val_accuracy: 0.8838 - lr: 0.0010


In [27]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Alpha droput 
for used to selu, if used regular dropout ill break the self normalized scaled. used aplha dropout to drop the neuron on layer have self normalized scaled

In [33]:
model_4 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.AlphaDropout(rate=2e-1),
    keras.layers.Dense(300,activation="selu",kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=2e-1),
    keras.layers.Dense(100,activation="selu",kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=2e-1),
    keras.layers.Dense((max(y_train)+1),activation="softmax",kernel_initializer="glorot_normal")
    
])
optimizer_sgd= keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
optimizer_nadam=keras.optimizers.Nadam(learning_rate=1e-3,beta_1=0.9,beta_2=0.999,)
model_4.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer_sgd,
                metrics=["accuracy"])
per_lr_sc = keras.callbacks.ReduceLROnPlateau(factor=5e-1, patience=5)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
check_point = keras.callbacks.ModelCheckpoint("droput_alpha.h5",save_best_only=True)
model_4.fit(X_train,y_train,epochs=20,
            validation_data =[X_valid,y_valid],
            callbacks=[per_lr_sc,tensorboard_cb,check_point])

Epoch 1/20
1719/1719 [==============================] - 5s 3ms/step - loss: 1.0660 - accuracy: 0.5978 - val_loss: 0.9327 - val_accuracy: 0.7048 - lr: 0.0100
Epoch 2/20
1719/1719 [==============================] - 4s 2ms/step - loss: 0.9167 - accuracy: 0.6557 - val_loss: 0.9712 - val_accuracy: 0.6942 - lr: 0.0100
Epoch 3/20
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8867 - accuracy: 0.6668 - val_loss: 0.8777 - val_accuracy: 0.7406 - lr: 0.0100
Epoch 4/20
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8732 - accuracy: 0.6733 - val_loss: 0.7488 - val_accuracy: 0.7398 - lr: 0.0100
Epoch 5/20
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8508 - accuracy: 0.6810 - val_loss: 0.8044 - val_accuracy: 0.7350 - lr: 0.0100
Epoch 6/20
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8421 - accuracy: 0.6824 - val_loss: 0.8752 - val_accuracy: 0.7176 - lr: 0.0100
Epoch 7/20
1719/1719 [==============================] - 4s

In [34]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2340), started 0:41:23 ago. (Use '!kill 2340' to kill it.)